This is a local, non-Kaggle notebook in which TFX 1.16.0 and python 3.10 and the compatible versions of other libraries are installed in a virtual environment that this notebook is running in.

paths are relative to the github repository directory, "recommender_systems"

In [1]:
import shutil

from tfx.orchestration import metadata

import tensorflow_transform as tft

from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store

import sys
import os
sys.path.append(os.path.join(os.getcwd(), "src/test/python/movie_lens_tfx"))
sys.path.append(os.path.join(os.getcwd(), "src/main/python/movie_lens_tfx"))

from helper import *
from movie_lens_tfx.PipelineComponentsFactory import *
from movie_lens_tfx.tune_train_movie_lens import *

from absl import logging
tf.get_logger().propagate = False
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

2025-11-04 16:49:09.522381: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:49:09.525394: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-04 16:49:09.567642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-04 16:49:10.438659: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:absl:tensorflow_io is not available: No module named 'tensorflow_io'
INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_decision_forests is

## EDA on the raw data

### w/ Polars and Plotly express
output is written to bin/local_notebook/images/

In [2]:
%run src/main/python/eda/eda_raw.py

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
key=ratings, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/ratings.dat
key=users, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/users.dat
key=movies, file_path=/home/nichole/projects/github/recommender_systems/src/main/resources/ml-1m/movies.dat
177 movies were not rated
0 users did not rate

ratings: DESCRIBE
<bound method DataFrame.describe of shape: (1_000_209, 4)
┌─────────┬──────────┬────────┬───────────┐
│ user_id ┆ movie_id ┆ rating ┆ timestamp │
│ ---     ┆ ---      ┆ ---    ┆ ---       │
│ i64     ┆ i64      ┆ i64    ┆ i64       │
╞═════════╪══════════╪════════╪═══════════╡
│ 1       ┆ 1193     ┆ 5      ┆ 978300760 │
│ 1       ┆ 661      ┆ 3      ┆ 978302109 │
│ 1       ┆ 914      ┆ 3      ┆ 978301968 │
│ 1       ┆ 3408     ┆ 4      ┆ 978300275 │
│ 1       ┆ 2355     ┆ 5      ┆ 978824291 │
│ …       ┆ …        ┆ …      ┆ …         │


### Run data pre-processing on full dataset to get the transformed data

In [3]:
infiles_dict_ser, output_config_ser, split_names = get_test_data(use_small=False)
user_id_max = 6040
movie_id_max = 3952
n_genres = N_GENRES
n_age_groups = N_AGE_GROUPS
n_occupations = 21
MIN_EVAL_SIZE = 50 #make this larger for production pipeline

test_num = "1"
    
PIPELINE_NAME = 'TestPipelines'
output_data_dir = os.path.join(get_bin_dir(), "local_notebook", test_num)
PIPELINE_ROOT = os.path.join(output_data_dir, PIPELINE_NAME)

# remove results from previous test runs:
try:
  print(f"removing: {PIPELINE_ROOT}")
  shutil.rmtree(PIPELINE_ROOT)
except OSError as e:
  pass
METADATA_PATH = os.path.join(PIPELINE_ROOT, 'tfx_metadata',
                             'metadata.db')
os.makedirs(os.path.join(PIPELINE_ROOT, 'tfx_metadata'),
            exist_ok=True)

ENABLE_CACHE = True

# metadata_connection_config = metadata_store_pb2.ConnectionConfig()
# metadata_connection_config.sqlite.SetInParent()
# metadata_connection = metadata.Metadata(metadata_connection_config)
metadata_connection_config = metadata.sqlite_metadata_connection_config(
  METADATA_PATH)

store = metadata_store.MetadataStore(metadata_connection_config)

if get_kaggle():
  tr_dir = "/kaggle/working/"
else:
  tr_dir = os.path.join(get_project_dir(), "src/main/python/movie_lens_tfx")

serving_model_dir = os.path.join(PIPELINE_ROOT, 'serving_model')
output_parquet_path = os.path.join(PIPELINE_ROOT, "transformed_parquet")

# for the custom ingestion component, the apache beam pipeline needs to be able to
# find the sibling scripts it imports.
# 2 solutions: (1) create a tar archive and use --extra_package in pipeline args
# or (2) use setup.py and --setup_file in pipeline args.

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0',
  '--setup_file=setup.py',
  #f'--extra_package={ingest_tar_file}'
]

CWD=/home/nichole/projects/github/recommender_systems, kaggle=False
removing: /home/nichole/projects/github/recommender_systems/bin/local_notebook/1/TestPipelines


In [4]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

tf.get_logger().propagate = False
import logging
logging.getLogger().setLevel(logging.WARNING)
from absl import logging
logging.set_verbosity(logging.WARNING)
logging.set_stderrthreshold(logging.WARNING)

context = InteractiveContext(pipeline_name=PIPELINE_NAME, pipeline_root=PIPELINE_ROOT,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args
)

factory = PipelineComponentsFactory(infiles_dict_ser, output_config_ser, tr_dir,
    user_id_max, movie_id_max, n_genres, n_age_groups,
    MIN_EVAL_SIZE, serving_model_dir, output_parquet_path)

components = factory.build_components(PIPELINE_TYPE.PREPROCESSING)

for component in components:
    context.run(component)

print(f'done pre-processing data')

UnboundLocalError: local variable 'example_resolver' referenced before assignment

## EDA on the transformed data
output is written to bin/local_notebook/images/

### using Polars, Plotly.express 

this can take an hour on a single COTS

In [ ]:
%run src/main/python/eda/eda_transformed.py

### using Pandas, Pyspark MLLIB FPGrowth

This does a market basket analysis with movie_ids.

If you want the PrefixSpan plots also, set
PLOT_PREFIXSPAN = True
but beware that the script will take much longer to run.

In [ ]:
PLOT_PREFIXSPAN=False

In [ ]:
%run src/main/python/eda/eda_transformed_pyspark_mllib.py

### Data and Concept Drift
After exploring the data inputs to the model, we want to define monitoring
for data and concept shifts.

let X = features

let Y = targets

Data shift is a change in the joint distribution, P(X, Y). 

Using the probability product rule, we can explore 4 causes for the
simplest changes in P(X,Y)
$$ P(X, Y) = P(Y, X) = P(X|Y)P(Y) = P(Y|X)P(X) $$

We can look for changes in one member in the following pairs at any time
(one is simpler than exploring more than 1 member changing at same time):
$$ P(X|Y) * P(Y) $$
$$ P(Y|X) * P(X) $$

* Covariate shift:
  $$ P(X) changed.  P(Y|X) unchanged $$
  Distr of model inputs changes.
* Label shift:
  $$ P(Y) changed,  P(X|Y) unchanged $$
  Distr of model outputs changes, but for any given output, the input distribution stays the same.
  
* Concept shift:
  $$ P(X) unchanged,  P(Y|X) changed $$
* Manifestation shift:
  $$ P(Y) unchanged,  P(X|Y) changed $$

[see more at NannyML](https://www.nannyml.com/blog/concept-drift)

### using TFDV to look at the raw data

In [ ]:
from tfx.dsl.io import fileio
from tfx.orchestration import metadata
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator
from tfx.utils import io_utils
from tensorflow_metadata.proto.v0 import anomalies_pb2, schema_pb2
from tensorflow_transform.tf_metadata import schema_utils


_list = store.get_artifacts_by_type("ExampleStatistics")
print(f'artifacts={_list}')
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "ExampleStatistics" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    stats_proto = tfdv.load_stats_binary(file_path)
    tfdv.visualize_statistics(stats_proto)

#ExampleAnomalies
_list = store.get_artifacts_by_type("ExampleAnomalies")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "ExampleAnomalies" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "SchemaDiff.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    stats_proto = tfdv.load_stats_binary(file_path)
    tfdv.visualize_artifacts(stats_proto)

_list = store.get_artifacts_by_type("ExampleStatistics")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "pre_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    stats_proto = tfdv.load_stats_binary(file_path)
    tfdv.visualize_statistics(stats_proto)
    

### using TFDV to look at the transformed data

In [ ]:

_list = store.get_artifacts_by_type("ExampleStatistics")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_stats" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "FeatureStats.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    stats_proto = tfdv.load_stats_binary(file_path)
    tfdv.visualize_statistics(stats_proto)
    
#ExampleAnomalies
_list = store.get_artifacts_by_type("ExampleAnomalies")
_list = sorted(_list, key=lambda x: x.create_time_since_epoch, reverse=True)
for artifact in _list:
    if "post_transform_anomalies" in artifact.uri:
        artifact_uri = artifact.uri
        break
assert(artifact_uri is not None)
file_paths = [os.path.join(artifact_uri, name, "SchemaDiff.pb") 
  for name in os.listdir(artifact_uri)]
for file_path in file_paths:
    stats_proto = tfdv.load_stats_binary(file_path)
    tfdv.visualize_artifacts(stats_proto)


## Run baseline model pipeline with full dataset

pipeline_factory = PipelineComponentsFactory(
  infiles_dict_ser=infiles_dict_ser, output_config_ser=output_config_ser,
  transform_dir=tr_dir, user_id_max=user_id_max, movie_id_max=movie_id_max,
  n_genres=n_genres, n_age_groups=n_age_groups, min_eval_size=MIN_EVAL_SIZE,
  serving_model_dir=serving_model_dir,
)

beam_pipeline_args = [
  '--direct_running_mode=multi_processing',
  '--direct_num_workers=0'
    ]

baseline_components = pipeline_factory.build_components(MODEL_TYPE.BASELINE)
    
# create baseline model
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=baseline_components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)

artifact_types = store.get_artifact_types()
logging.debug(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
logging.debug(f"MLMD store artifacts={artifacts}")

components = pipeline_factory.build_components(MODEL_TYPE.PRODUCTION)
# simulate experimentation of one model family
my_pipeline = tfx.dsl.Pipeline(
  pipeline_name=PIPELINE_NAME,
  pipeline_root=PIPELINE_ROOT,
  components=components,
  enable_cache=ENABLE_CACHE,
  metadata_connection_config=metadata_connection_config,
  beam_pipeline_args=beam_pipeline_args,
)

tfx.orchestration.LocalDagRunner().run(my_pipeline)


artifact_types = store.get_artifact_types()
print(f"MLMD store artifact_types={artifact_types}")
artifacts = store.get_artifacts()
print(f"MLMD store artifacts={artifacts}")

executions = store.get_executions()
logging.debug(f"MLMD store executions={executions}")

# executions has custom_properties.key: "infiles_dict_ser"
#    and custom_properties.key: "output_config_ser"
artifact_count = len(artifacts)
execution_count = len(executions)
